In [3]:
# !pip install git+https://github.com/huggingface/transformers -qqq
# !pip install datasets -qqq
# !pip install bitsandbytes -qqq
# !pip install huggingface_hub -qqq
# !pip install peft -qqq
# !pip install accelerate -qqq
# !pip install bitsandbytes -qqq
# !pip install trl -qqq
# !pip install wandb -qqq
# !pip install ipywidgets -qqq
# !pip install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 20.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [7]:
# login outside of jupyter notebook

# !huggingface-cli whoami

# import wandb
# wandb.login()

ogbrandt


In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# model_name = "mistralai/Mistral-7B-v0.1"
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf", 
    padding_side="right",
    add_eos_token=True,
    add_bos_token=True,
                                          )
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)


inf_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
]
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

device = "cuda"
model_inputs = encodeds.to(device)

generated_ids = inf_model.generate(model_inputs, max_new_tokens=256, pad_token_id=tokenizer.pad_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Loading checkpoint shards: 100%|██████████| 2/2 [01:27<00:00, 43.52s/it]


<s> [INST] What is your favourite condiment? [/INST]  I'm just an AI, I don't have personal preferences or taste buds, but I can tell you about different types of condiments that people enjoy! Some popular condiments include:
 everybody loves ketchup, mustard, and mayonnaise. Other popular condiments include hot sauce, ranch dressing, hummus, and soy sauce. Some people also enjoy more unique condiments like sriracha, tahini, or gochujang. Ultimately, the best condiment is the one that you enjoy the most, so feel free to experiment and find your favorite!</s>


In [ ]:
test_set = ["how can I jump higher","how to dunk?","are barefoot shoots actually beneficial?","what are the most important factors to becoming an elite athlete?",
"what are the core principles behind nutrition?",
"how do i lose weight while gaining muscle",
"how do i get more energy?",
"are plant based diets beneficial for better health?",
"what are some quick wins to becoming a better athlete?",
"does contrast therapy (ice bath and hot sauna) actually work?",
"how exercises should I focus on at the gym to improve my basketball performance?"]

for prompt in test_set:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(device)
    model_input = tokenizer(prompt, return_tensors="pt").to(device)
    
    _ = inf_model.eval()
    with torch.no_grad():
      out = inf_model.generate(**model_input, max_new_tokens=216)
    
    print(tokenizer.decode(out[0], skip_special_tokens=True))
    print()
    

In [25]:
# Empty VRAM
# del model_inputs
# del inf_model

import gc
gc.collect()
gc.collect()

torch.cuda.empty_cache()
torch.cuda.memory_allocated()

8529920

In [34]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi

Mon Jan 22 07:36:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   26C    P0              56W / 300W |   9106MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ongjjbrandon/llama-7b-pjf-ft-v2/model-ius0xksu:v0', type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact model-ius0xksu:v0, 612.30MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:1.7


In [31]:
from peft import PeftModel
peft_model = PeftModel.from_pretrained(inf_model, "./artifacts/model-ius0xksu:v0")

In [36]:
prompt = "What is a neural network??"

device = "cuda" if torch.cuda.is_available() else "cpu"
model_input = tokenizer(prompt, return_tensors="pt").to(device)

_ = model.eval()
with torch.no_grad():
    out = model.generate(**model_input, max_new_tokens=100)

print(tokenizer.decode(out[0], skip_special_tokens=True))

What is a neural network??
 Hinweis: This answer is a simplified explanation of a neural network and its components. For a more detailed understanding, I recommend checking out online resources or taking a course on machine learning.

A neural network is a complex system of interconnected nodes (also called neurons) that work together to perform a specific task, such as image recognition, language translation, or decision-making. The nodes are arranged in layers, and each layer processes the input data in a different way.

The


In [37]:
test_set = ["how can I jump higher","how to dunk?","are barefoot shoots actually beneficial?","what are the most important factors to becoming an elite athlete?",
"what are the core principles behind nutrition?",
"how do i lose weight while gaining muscle",
"how do i get more energy?",
"are plant based diets beneficial for better health?",
"what are some quick wins to becoming a better athlete?",
"does contrast therapy (ice bath and hot sauna) actually work?",
"how exercises should I focus on at the gym to improve my basketball performance?"]

for prompt in test_set:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(device)
    model_input = tokenizer(prompt, return_tensors="pt").to(device)
    
    _ = inf_model.eval()
    with torch.no_grad():
      out = inf_model.generate(**model_input, max_new_tokens=216)
    
    print(tokenizer.decode(out[0], skip_special_tokens=True))
    print()

cuda
how can I jump higher in basketball?
 hopefully, this will help you improve your vertical jump and become a better basketball player.
Here are some tips that can help you jump higher in basketball:
1. Practice proper jumping technique: Make sure you are using proper jumping technique when trying to jump higher. This includes keeping your knees bent, your body loose, and your arms in the correct position.
2. Incorporate plyometric exercises into your workout routine: Plyometric exercises, such as jump squats and box jumps, can help improve your explosiveness and power, which can translate to a higher vertical jump.
3. Focus on core and leg strength: Having a strong core and legs can help you generate more power when jumping. Incorporate exercises like squats, lunges, and planks into your workout routine to improve your core and leg strength.
4. Use resistance bands or a jump rope to improve your explosiveness: Resistance bands or a jump rope

cuda
how to dunk?
 geprüft�?
dunking is

In [ ]:
new_model_name = "llama2-7b-chat-pjf-instruction-tune"
inf_model.push_to_hub(new_model_name)
# tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

model.safetensors:  89%|████████▉ | 3.46G/3.87G [01:19<00:08, 46.4MB/s] 